## Гипотезы о долях

In [1]:
import pandas as pd
import numpy as np
data = pd.read_csv(r'C:\MyStorage\Self-Education\Coursera\Specialization\Data Structures and Algorithms\2.Data Structures\Week 2\banknotes.txt',sep = '\t')

In [4]:
is_real = data['real']
measures = data.drop('real', axis=1)

In [6]:
from sklearn.cross_validation import train_test_split
measures_train, measures_test, is_real_train, is_real_test = train_test_split(measures, is_real, test_size=0.25, random_state=1)

In [11]:
measures.columns

Index([u'X1', u'X2', u'X3', u'X4', u'X5', u'X6'], dtype='object')

In [13]:
from sklearn.linear_model import LogisticRegression
model1 = LogisticRegression()
model1.fit(measures_train[['X1','X2','X3']], is_real_train)
model2 = LogisticRegression()
model2.fit(measures_train[['X4','X5','X6']], is_real_train)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [23]:
answer1 = model1.predict(measures_test[['X1','X2','X3']])
1-np.mean(np.abs(is_real_test - model1.predict(measures_test[['X1','X2','X3']])))

0.8

In [22]:
answer2 = model2.predict(measures_test[['X4','X5','X6']])
1-np.mean(np.abs(is_real_test - model2.predict(measures_test[['X4','X5','X6']])))

0.98

In [27]:
import scipy
def proportions_diff_z_stat_rel(sample1, sample2):
    sample = zip(sample1, sample2)
    n = len(sample)
    
    f = sum([1 if (x[0] == 1 and x[1] == 0) else 0 for x in sample])
    g = sum([1 if (x[0] == 0 and x[1] == 1) else 0 for x in sample])
    
    return float(f - g) / np.sqrt(f + g - float((f - g)**2) / n )

def proportions_diff_z_test(z_stat, alternative = 'two-sided'):
    if alternative not in ('two-sided', 'less', 'greater'):
        raise ValueError("alternative not recognized\n"
                         "should be 'two-sided', 'less' or 'greater'")
    
    if alternative == 'two-sided':
        return 2 * (1 - scipy.stats.norm.cdf(np.abs(z_stat)))
    
    if alternative == 'less':
        return scipy.stats.norm.cdf(z_stat)

    if alternative == 'greater':
        return 1 - scipy.stats.norm.cdf(z_stat)

In [28]:
proportions_diff_z_test(proportions_diff_z_stat_rel(answer1, answer2))

0.12282264810139254

In [29]:
def proportions_diff_confint_rel(sample1, sample2, alpha = 0.05):
    z = scipy.stats.norm.ppf(1 - alpha / 2.)
    sample = zip(sample1, sample2)
    n = len(sample)
        
    f = sum([1 if (x[0] == 1 and x[1] == 0) else 0 for x in sample])
    g = sum([1 if (x[0] == 0 and x[1] == 1) else 0 for x in sample])
    
    left_boundary = float(f - g) / n  - z * np.sqrt(float((f + g)) / n**2 - float((f - g)**2) / n**3)
    right_boundary = float(f - g) / n  + z * np.sqrt(float((f + g)) / n**2 - float((f - g)**2) / n**3)
    return (left_boundary, right_boundary)

In [30]:
proportions_diff_confint_rel(answer1, answer2)

(-0.22702018362022364, 0.027020183620223626)